<a href="https://colab.research.google.com/github/Quddos/machine-learning/blob/main/Offline_Digital_Signature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 1s (311 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
# Colab-ready starter: extract crops from scanned PDF sheets
# Requires: pip install pdf2image opencv-python-headless pillow numpy pandas scikit-image
!pip install -q pdf2image opencv-python-headless pillow numpy pandas scikit-image

from pdf2image import convert_from_path
import cv2, numpy as np, os, math
from PIL import Image
import pandas as pd

# === PARAMETERS ===
PDF_PATH = "/content/offlinesignature.pdf"   # upload the file to Colab or mount Drive
OUT_DIR = "/content/signature_crops"
DPI = 300
MIN_BOX_AREA = 2000   # tune if small boxes are missed
os.makedirs(OUT_DIR, exist_ok=True)

# Convert PDF -> images
pages = convert_from_path(PDF_PATH, dpi=DPI)
print(f"Converted {len(pages)} pages")

manifest = []  # rows: dict with filename, page_idx, box_idx, bbox

def auto_rotate_if_needed(img_cv):
    # Try to detect predominant text orientation by Hough lines / or use minAreaRect of non-white area
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    _,th = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV)
    coords = cv2.findNonZero(th)
    if coords is None:
        return img_cv
    rect = cv2.minAreaRect(coords)
    angle = rect[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    # rotate if angle significant
    if abs(angle) > 2:
        (h, w) = img_cv.shape[:2]
        M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
        rotated = cv2.warpAffine(img_cv, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        return rotated
    return img_cv

for i, page_pil in enumerate(pages):
    page_idx = i+1
    # convert to OpenCV image (RGB->BGR)
    img = cv2.cvtColor(np.array(page_pil), cv2.COLOR_RGB2BGR)

    # rotate if needed (your pages appear rotated) — this attempts auto-orientation
    img = auto_rotate_if_needed(img)

    # convert to grayscale, blur and threshold
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # increase contrast slightly
    gray = cv2.equalizeHist(gray)

    # Remove thin lines and keep boxes: morphological ops
    # First, binary threshold (adaptive helps if illumination varies)
    th = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 15, 8)

    # Morphological closing to join box lines & signature strokes
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    th_closed = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(th_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Filter contours by area and aspect ratio to find boxes
    boxes = []
    h, w = gray.shape
    for cnt in contours:
        x,y,ww,hh = cv2.boundingRect(cnt)
        area = ww*hh
        if area < MIN_BOX_AREA:
            continue
        # likely box: avoid page border and left margin (where name/ID are)
        if x < w*0.05 and ww > w*0.6:
            # skip large left margin artifacts
            continue
        # Some small filtering by aspect ratio
        ar = ww/float(hh)
        if ar < 0.4 or ar > 4.0:
            # But allow wide boxes (some scans may combine)
            pass
        boxes.append((x,y,ww,hh,area))

    # Heuristic: if boxes fewer than expected, try dilating more aggressively
    if len(boxes) < 10:
        kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
        th2 = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel2, iterations=2)
        contours2, _ = cv2.findContours(th2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        boxes = []
        for cnt in contours2:
            x,y,ww,hh = cv2.boundingRect(cnt)
            if ww*hh < MIN_BOX_AREA: continue
            if x < w*0.05 and ww > w*0.6: continue
            boxes.append((x,y,ww,hh,ww*hh))

    # Sort boxes top->bottom, left->right
    boxes = sorted(boxes, key=lambda b: (b[1], b[0]))

    # Optionally: attempt grid clustering to order boxes in row-major order
    # Crop and save
    page_out = os.path.join(OUT_DIR, f"page_{page_idx}")
    os.makedirs(page_out, exist_ok=True)

    for bi, (x,y,ww,hh,area) in enumerate(boxes):
        pad = 6  # small padding
        x0 = max(0, x-pad); y0 = max(0, y-pad)
        x1 = min(w, x+ww+pad); y1 = min(h, y+hh+pad)
        crop = img[y0:y1, x0:x1]
        fname = f"page{page_idx:02d}_box{bi:03d}.png"
        out_path = os.path.join(page_out, fname)
        cv2.imwrite(out_path, crop)
        manifest.append({
            "file": out_path,
            "page": page_idx,
            "box_idx": bi,
            "x": x0, "y": y0, "w": x1-x0, "h": y1-y0,
            "person_id": "",   # fill later (manual/automated)
            "label": ""        # "genuine" or "forgery" - fill later
        })

    print(f"Page {page_idx}: found {len(boxes)} boxes, saved to {page_out}")

# Save manifest CSV for manual labeling
df = pd.DataFrame(manifest)
csv_path = os.path.join(OUT_DIR, "manifest.csv")
df.to_csv(csv_path, index=False)
print("Manifest saved:", csv_path)


Converted 20 pages
Page 1: found 34 boxes, saved to /content/signature_crops/page_1
Page 2: found 39 boxes, saved to /content/signature_crops/page_2
Page 3: found 35 boxes, saved to /content/signature_crops/page_3
Page 4: found 36 boxes, saved to /content/signature_crops/page_4
Page 5: found 38 boxes, saved to /content/signature_crops/page_5
Page 6: found 30 boxes, saved to /content/signature_crops/page_6
Page 7: found 36 boxes, saved to /content/signature_crops/page_7
Page 8: found 39 boxes, saved to /content/signature_crops/page_8
Page 9: found 35 boxes, saved to /content/signature_crops/page_9
Page 10: found 32 boxes, saved to /content/signature_crops/page_10
Page 11: found 33 boxes, saved to /content/signature_crops/page_11
Page 12: found 29 boxes, saved to /content/signature_crops/page_12
Page 13: found 44 boxes, saved to /content/signature_crops/page_13
Page 14: found 36 boxes, saved to /content/signature_crops/page_14
Page 15: found 33 boxes, saved to /content/signature_crops/pa

Automatic Labeling

In [6]:

















import pandas as pd

# Load the manifest you already generated
df = pd.read_csv("/content/signature_crops/manifest.csv")

# Auto assign person IDs (P001, P002, …)
pages = sorted(df['page'].unique())
person_ids = {page: f"P{page:03d}" for page in pages}

df['person_id'] = df['page'].map(person_ids)

# All signatures are genuine
df['label'] = "genuine"

# Save labeled manifest
output_path = "/content/signature_crops/manifest_labeled.csv"
df.to_csv(output_path, index=False)

print("✔ Automatic labeling complete!")
print("Saved:", output_path)
df.head()


✔ Automatic labeling complete!
Saved: /content/signature_crops/manifest_labeled.csv


,file,page,box_idx,x,y,w,h,person_id,label
0,/content/signature_crops/page_1/page01_box000.png,1,0,492,6,329,664,P001,genuine
1,/content/signature_crops/page_1/page01_box001.png,1,1,829,10,321,663,P001,genuine
2,/content/signature_crops/page_1/page01_box002.png,1,2,1166,13,324,664,P001,genuine
3,/content/signature_crops/page_1/page01_box003.png,1,3,1507,16,321,664,P001,genuine
4,/content/signature_crops/page_1/page01_box004.png,1,4,1843,19,324,664,P001,genuine


from matplotlib import pyplot as plt
_df_0['box_idx'].plot(kind='hist', bins=20, title='box_idx')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['x'].plot(kind='hist', bins=20, title='x')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['y'].plot(kind='hist', bins=20, title='y')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['file'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='x', y='file', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['file'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='y', y='file', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['file'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='w', y='file', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [7]:
import os, random
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import defaultdict

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Load the labeled manifest
df = pd.read_csv("/content/signature_crops/manifest_labeled.csv")

print("Total signature images:", len(df))
print("Unique persons:", df['person_id'].nunique())
df.head()


Total signature images: 710
Unique persons: 20


,file,page,box_idx,x,y,w,h,person_id,label
0,/content/signature_crops/page_1/page01_box000.png,1,0,492,6,329,664,P001,genuine
1,/content/signature_crops/page_1/page01_box001.png,1,1,829,10,321,663,P001,genuine
2,/content/signature_crops/page_1/page01_box002.png,1,2,1166,13,324,664,P001,genuine
3,/content/signature_crops/page_1/page01_box003.png,1,3,1507,16,321,664,P001,genuine
4,/content/signature_crops/page_1/page01_box004.png,1,4,1843,19,324,664,P001,genuine


In [8]:
# Group images under each person_id
groups = defaultdict(list)

for i, row in df.iterrows():
    pid = row['person_id']
    groups[pid].append(row['file'])

# Print summary
for pid, imgs in list(groups.items())[:5]:
    print(pid, "->", len(imgs), "images")

print("Total persons:", len(groups))


P001 -> 34 images
P002 -> 39 images
P003 -> 35 images
P004 -> 36 images
P005 -> 38 images
Total persons: 20


In [9]:
def create_pairs(groups, pairs_per_person=40):
    pos_pairs = []
    neg_pairs = []
    person_list = list(groups.keys())

    for pid in person_list:
        imgs = groups[pid]

        # Positive pairs (same person)
        if len(imgs) >= 2:
            for _ in range(pairs_per_person):
                a, b = random.sample(imgs, 2)
                pos_pairs.append((a, b, 1))

        # Negative pairs (different persons)
        for _ in range(pairs_per_person):
            a = random.choice(imgs)
            other_pid = random.choice([p for p in person_list if p != pid])
            b = random.choice(groups[other_pid])
            neg_pairs.append((a, b, 0))

    all_pairs = pos_pairs + neg_pairs
    random.shuffle(all_pairs)
    return all_pairs

pairs = create_pairs(groups, pairs_per_person=40)
print("Total training pairs:", len(pairs))
print("Sample pair:", pairs[0])


Total training pairs: 1600
Sample pair: ('/content/signature_crops/page_19/page19_box001.png', '/content/signature_crops/page_15/page15_box003.png', 0)


In [10]:
IMG_SIZE = (224, 224)
AUTOTUNE = tf.data.AUTOTUNE
BATCH = 32

def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

def generator():
    for a, b, label in pairs:
        yield a.encode(), b.encode(), np.float32(label)

dataset = tf.data.Dataset.from_generator(
    generator,
    output_signature=(
        tf.TensorSpec([], tf.string),
        tf.TensorSpec([], tf.string),
        tf.TensorSpec([], tf.float32)
    )
)

def preprocess(a, b, label):
    return (load_image(a), load_image(b)), label

dataset = dataset.map(preprocess, num_parallel_calls=AUTOTUNE)

total = len(pairs)
train_ds = dataset.take(int(total * 0.8)).shuffle(1024).batch(BATCH).prefetch(AUTOTUNE)
val_ds   = dataset.skip(int(total * 0.8)).batch(BATCH).prefetch(AUTOTUNE)

print("Train batches:", tf.data.experimental.cardinality(train_ds).numpy())
print("Validation batches:", tf.data.experimental.cardinality(val_ds).numpy())


Train batches: -2
Validation batches: -2


In [11]:
from tensorflow.keras import layers, Model
import tensorflow as tf

def build_embedding_model(embedding_dim=128):
    # Base CNN (ImageNet pretrained)
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling='avg'
    )

    # Freeze base for initial training
    base.trainable = False

    inp = layers.Input(shape=(224,224,3))

    # EfficientNet preprocessing
    x = tf.keras.applications.efficientnet.preprocess_input(inp)

    # Extract features
    x = base(x, training=False)

    # Add embedding layers
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(embedding_dim)(x)

    # L2 normalize for stable similarity learning
    out = tf.math.l2_normalize(x, axis=-1)

    model = Model(inp, out, name="signature_encoder")
    return model

# Create encoder
encoder = build_embedding_model()
encoder.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
